# Insert homology arms

Each insert contains a 5' and 3' homology arm which include a KpnI and EcoRI recognition site respectively. These sequnces come directly from pFC9 and in this notebook I am creating genbank files of the desired regions and checking to make sure they have the features required.

In [187]:
import os
from pydna.readers import readsembly
from pydna.design import primer_design
from pydna.design import assembly_fragments
from pydna.dseqrecord import Dseqrecord
from Bio.Restriction import *
from pydna.amplicon import Amplicon
from Bio.Seq import Seq
from Bio import SeqIO
import yaml
import pprint

In [188]:
pFC9_path = '../resources/files/genbank/pFC9.gb'
pFC9 = read(pFC9_path)

In [189]:
arm_length = 30

## 5' arm

The 5' arm is composed of the 30 nucleotides upstream of the KnpI recognition site.

First make sure KpnI is in fact a unique cutter.

In [190]:
cut_location = KpnI.search(pFC9.seq)
assert len(cut_location) == 1
print(cut_location)

[56]


Locate the recognition site in the sequence. This gives the start coordinate of the recognition site.

In [191]:
kpnI_start = pFC9.seq.find(KpnI.site)
kpnI_start

50

Get the end position. Check to make sure the sequence at the start and end coordinates match the recognition site sequence.

In [192]:
kpnI_end = kpnI_start + len(KpnI.site)
assert str(pFC9.seq[kpnI_start:kpnI_end]) == str(KpnI.site)

Get the homology arm sequence.

In [193]:
five_arm_start = kpnI_end-arm_length
five_arm_end = kpnI_end

In [194]:
five_arm = pFC9.seq[five_arm_start:five_arm_end]
assert len(five_arm) == arm_length
five_arm

Dseq(-30)
TAGGGCGAATTGGAGCTCCACATCGGTACC
ATCCCGCTTAACCTCGAGGTGTAGCCATGG

Double check to make sure last 6 nucleotides are the KpnI site.

In [195]:
assert str(five_arm[-6:]) == str(KpnI.site)
print('These must match')
print('Last 6 5 arm:', five_arm[-6:])
print('KnpI site:', KpnI.site)

These must match
Last 6 5 arm: GGTACC
KnpI site: GGTACC


Show the digestion.

In [196]:
five_arm.cut(KpnI)

(Dseq(-29)
 TAGGGCGAATTGGAGCTCCACATCGGTAC
 ATCCCGCTTAACCTCGAGGTGTAGC,
 Dseq(-5)
     C
 CATGG)

## 3' arm

The 3' arm is composed of the EcoRI recognition site and 24 downstream nucleotides.

Again find the location of the EcoRI site in pFC9 and confirm it is unique.

In [197]:
eco_cut_location = EcoRI.search(pFC9.seq)
assert len(eco_cut_location) == 1
print(eco_cut_location)

[541]


Get start and end positions of the recognition site.

In [198]:
EcoRI_start = pFC9.seq.find(EcoRI.site)
EcoRI_start

539

In [199]:
EcoRI_end = EcoRI_start + len(EcoRI.site)

In [200]:
assert str(pFC9.seq[EcoRI_start: EcoRI_end]) == EcoRI.site

Get nucleotides **downstream** of the EcoRI recognition site to complete the homology arm.

In [201]:
three_arm_start = EcoRI_start
three_arm_end = EcoRI_start + arm_length
three_arm = pFC9.seq[three_arm_start:three_arm_end]
assert len(three_arm) == arm_length
three_arm

Dseq(-30)
GAATTCGTCGCAGTGACCGAGGCGAGGAGG
CTTAAGCAGCGTCACTGGCTCCGCTCCTCC

Check that 3' arm begins with the recognition sequence.

In [202]:
assert three_arm.find(EcoRI.site) == 0

Show digestion

In [203]:
three_arm.cut(EcoRI)

(Dseq(-5)
 G
 CTTAA,
 Dseq(-29)
 AATTCGTCGCAGTGACCGAGGCGAGGAGG
     GCAGCGTCACTGGCTCCGCTCCTCC)

## Write output files

Write genbank formated files for both homology arms.

In [204]:
from pydna.genbankrecord import GenbankRecord
import datetime

In [205]:
def write_seq_as_genbank(record, name, output_path, version=1.0, **kwargs):
    g_record = GenbankRecord(record)
    g_record.locus=name.replace(' ', '_')
    g_record.id = f'v{version}'
    g_record.stamp()
    g_record.add_feature(0, len(record), name=name, **kwargs)
    g_record.write(output_path)

Write genbank files for 5' and 3' arms.

In [206]:
author = 'Ethan Holleman'
note_5 = f'Homology arm of variable region insert taken from (base 1) positions {five_arm_start+1}-{five_arm_end+1} of pFC9'
note_3 = f'Homology arm of variable region insert taken from (base 1) positions {three_arm_start+1}-{three_arm_end+1} of pFC9'

In [207]:
mod_date = datetime.date.strftime(datetime.datetime.now(), "%m/%d/%Y")

In [208]:
write_seq_as_genbank(
    five_arm, 
    "5' homology arm", 
    '../resources/files/genbank/5_prime_homology_arm.gb', 
    author=author,
    label='5_prime_HR',
    note=note_5,
    pFC9_start=five_arm_start,
    pFC9_end=five_arm_end,
    pFC9_seguid=pFC9.seq.seguid(),
    date=mod_date
)

../resources/files/genbank/5_prime_homology_arm.gb

In [209]:
write_seq_as_genbank(
    three_arm, 
    "3' homology arm", 
    '../resources/files/genbank/3_prime_homology_arm.gb', 
    author=author,
    label='3_prime_HR',
    note=note_3,
    pFC9_start=three_arm_start,
    pFC9_end=three_arm_end,
    pFC9_seguid=pFC9.seq.seguid(),
    date=mod_date
)

../resources/files/genbank/3_prime_homology_arm.gb

## Testing insert

Testing insertion into initiation construct.

Cut pFC9 with KpnI and EcoRI.

In [210]:
pFC9_linear = pFC9.cut((KpnI, EcoRI))
pFC9_lf = max(pFC9_linear, key=lambda x: len(x))
pFC9_lf

Dseqrecord(-3104)

Simulate an insert.

In [211]:
def simulate_t5(seq):
    # arbitrary say degrades 35 nucleotides
    seq.watson = seq.watson[35:]
    seq.crick = seq.crick[:-35]
    return seq


In [221]:
insert = read('../resources/files/genbank/5_prime_homology_arm.gb') + read('../resources/files/genbank/3_prime_homology_arm.gb')


In [223]:
pFC9_lf = Dseqrecord(pFC9_lf)
pFC9_lf.name = 'pFC9'
insert = Dseqrecord(insert)
insert.name = 'insert'
fragments = [insert, pFC9_lf]
a = Assembly(fragments)
a.assemble_circular()[0].write()

Sequence changed. new: name.gb     size: 3106bp topology: o SEGUID: k_JxCZbmMDU5QaHsp7BTgH3zJGU old: name_OLD.gb size: 3146bp topology: o SEGUID: NU8CyuEklkcigVDHu-LlDYO5yWY

In [226]:
def help_me(*args):
    print(args)
help_me(*[1, 2, 3], *[4, 5, 6])

(1, 2, 3, 4, 5, 6)


In [ ]:
d = Dseqrecord